In [1]:
import pathlib

import numpy as np
import polars as pl
import pandas as pd
import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
rg_paths = sorted(pathlib.Path("data/rg").glob("*.HEreg"))

print(len(rg_paths))

349497


In [3]:
usable_rg_dfs = list()

for path in tqdm.tqdm(rg_paths):
    rg_df = pd.read_csv(path, sep="\s+", skiprows=2, index_col=0)
    if np.isnan(rg_df.loc["rG", "Estimate"]):
        continue

    pheno1, pheno2 = path.stem.replace("rg_sub_0_", "").split("_")

    rg_df = (
        rg_df
        .loc[["V(G)/Vp_tr1", "V(G)/Vp_tr2", "C(G)/Vp_tr12", "rG", "N_tr1", "N_tr2"], :]
        .assign(
            pheno1=pheno1,
            pheno2=pheno2
        )
    )
    
    usable_rg_dfs.append(rg_df)

usable_rg_df = pd.concat(usable_rg_dfs)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 349497/349497 [09:49<00:00, 592.86it/s]


In [6]:
usable_rg_df.reset_index().pipe(pl.DataFrame).write_parquet("data/gathered_rg.parquet")

In [7]:
usable_rg_df.head(2)

,Estimate,SE_OLS,SE_Jackknife,P_OLS,P_Jackknife,pheno1,pheno2
Coefficient,,,,,,,
V(G)/Vp_tr1,0.023622,0.016090,0.022798,0.142077,0.300132,1400,1405
V(G)/Vp_tr2,0.021113,0.015663,0.029673,0.177689,0.476774,1400,1405
